In [60]:
#Imports
from sgp4.api import jday
import datetime
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
%matplotlib notebook
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from sklearn.neighbors import KernelDensity

In [61]:
# Read files
data_cosmos_satellite_garbage = pd.read_csv('../Data/cosmos.csv')
data_cosmos_satellite_iridium = pd.read_csv('../Data/iridium.csv')

In [62]:
#Get date
current_time = datetime.datetime.now()
xd, xr = jday(current_time.year , current_time.month , current_time.day, current_time.hour, current_time.minute , current_time.second)
print(xd)
print(xr)

2460356.5
0.8780439814814814


In [63]:
# DBSCAN for cosmos garbage
with open('../Data/cosmos.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Cosmos_G = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Cosmos_G.append(r)

data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers')])
fig.show()


model = DBSCAN(eps=100, min_samples=2)
model.fit_predict(data_garbage_cosmos)
pred = model.fit_predict(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers', marker={"size":12, "color":pred})])
fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)


number of cluster found: 9
cluster for each point:  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [64]:
# DBSCAN for iridium garbage
with open('../Data/iridium.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Iridium_G = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Iridium_G.append(r)

data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers')])
fig.show()

model = DBSCAN(eps=1000, min_samples=2)
model.fit_predict(data_garbage_iridium)
pred = model.fit_predict(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers', marker={"size":12, "color":pred})])

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 32
cluster for each point:  [ 0  1 -1 -1  2  3  4  5  6  7  8  9 -1  6 -1 -1 10 11 12 13 -1 14 15 16
 -1  1 -1 -1 17 18 11  1 19  1 -1 -1 16 20 -1 21 22 15 23 -1 -1 -1 -1 22
 -1 24  6 -1 -1 -1 -1 -1  9 25  2 26 23 25  6 -1 -1 27 -1 28 -1 -1  2 -1
  2  8  6  2 -1 15 20 -1  8 13 -1 17  8 11 -1 -1  1 10 -1 -1 -1 -1  5 29
 27 -1 12  8 28 15  9 24  6  8 10 10 28 -1  0  7  4 30 -1 -1  8  8 -1 -1
 -1 10  8 13 21  1 -1 -1 18 -1  8  1 10 19 -1  6 -1 26  8 -1 24 27 27 -1
 -1 29 24  8  4  2  5 -1  8 -1 -1  1 25 -1 -1  3  8 30 14 22 28 13 27 -1]


In [65]:
#KDE for iridium garbage
data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]

model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_iridium)
log_dens = model.score_samples(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers', marker={"size":12, "color":log_dens})])
fig.show()

In [66]:
#KDE for cosmos garbage
data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_cosmos)
log_dens = model.score_samples(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers', marker={"size":12, "color":log_dens})])
fig.show()